In [ ]:
import scGAM as scGAM
import gseapy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import numpy as np
import scanpy.external as sce

In [ ]:
from pygam import LinearGAM, s
import numpy as np
import pandas as pd
import time
from collections import OrderedDict
from joblib import delayed, Parallel
from sklearn.preprocessing import StandardScaler
import scanpy as sc
import seaborn as sns

In [ ]:
sc.settings.verbosity = 3  
# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=True, figsize=(5, 5))

In [ ]:
# samples_dict = {"Immature": "A0021_043", "0hr": "A0008_041", "1hr": "A0021_044", "4hr": "A0008_045", "4hr_replicate": "A0021_042", "6hr": "A0021_045", "8hr": "A0008_044", "8hr_replicate": "A0021_038", "11hr": "A0008_046", "12hr": "A0021_046"}
samples_dict = {"Immature": "A0021_043", "0hr": "A0008_041", "1hr": "A0021_044", "4hr": "A0021_042", "6hr": "A0021_045", "8hr": "A0008_044", "11hr": "A0008_046", "12hr": "A0021_046"}

In [ ]:
s_genes = list(pd.read_csv("./../../MouseInfection/s.genes.txt")["x"])
g2m_genes = list(pd.read_csv("./../../MouseInfection/g2m.genes.txt")["x"])
print(s_genes)
print(g2m_genes)

In [ ]:
adata_ovary_combined = sc.read_h5ad("./../../pyobjs_github/adata_ovary_combined_processed_deconvoluted_newscaling.h5ad")
adata_ovary_combined.uns['log1p'] = {'base' : None}
adata_ovary_combined.shape

In [ ]:
granulosa = sc.read_h5ad("./../../pyobjs_github/granulosa1.h5ad")
granulosa.uns['log1p'] = {'base' : None}
granulosa.shape

In [ ]:
adata_ovary_combined.obs["granulosa_celltypes_reduced"] = adata_ovary_combined.obs["Level0_max_pred_celltype"]
adata_ovary_combined.obs["granulosa_celltypes_reduced"] = adata_ovary_combined.obs["granulosa_celltypes_reduced"].astype(str)
adata_ovary_combined.obs.loc[granulosa.obs_names,"granulosa_celltypes_reduced"] = granulosa.obs["granulosa_celltypes_reduced"]
adata_ovary_combined.obs["granulosa_celltypes_reduced"] = adata_ovary_combined.obs["granulosa_celltypes_reduced"].astype("category")

In [ ]:
del adata_ovary_combined

In [ ]:
granulosa_mural = sc.read_h5ad("./../../pyobjs_github/granulosa_mural.h5ad")
granulosa_mural.uns['log1p']["base"] = None
granulosa_mural.shape

In [ ]:
granulosa_temp = granulosa_mural[granulosa_mural.obs["Sample"] == "Immature"]
sc.tl.rank_genes_groups(granulosa_temp, groupby="granulosa_celltypes_follicle_type", method="wilcoxon", key_added="rank_gene_groups_granulosa_celltypes_follicle_type")
sc.tl.rank_genes_groups(granulosa_temp, groupby="granulosa_celltypes_reduced", key_added="rank_gene_groups_granulosa_celltypes_reduced")

In [ ]:
result = granulosa_temp.uns['rank_gene_groups_granulosa_celltypes_follicle_type']
groups = result['names'].dtype.names
dge_results = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names', "logfoldchanges", "pvals_adj"]})
marker_list = list()
for group in result['names'].dtype.names:
    markers = sc.get.rank_genes_groups_df(granulosa_temp, key='rank_gene_groups_granulosa_celltypes_follicle_type', group = group)
    markers["celltypes"] = group
    marker_list.append(markers)
df = marker_list[1]
df = df.set_index("names")
df.to_csv("./../pyobjs/Immature_rank_gene_groups_granulosa_celltypes_reduced.csv")

In [ ]:
sc.tl.dendrogram(granulosa_temp, groupby="granulosa_celltypes_reduced")
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=6, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
sc.pl.rank_genes_groups_matrixplot(granulosa_temp, key="rank_gene_groups_granulosa_celltypes_follicle_type", n_genes=10, cmap='RdBu_r', values_to_plot="logfoldchanges", min_logfoldchange=1.0, save="rank_gene_groups_granulosa_celltypes_follicle_type", figsize = (6,0.4))
sc.settings.set_figure_params(dpi=80, facecolor='white', frameon=True, figsize=(2, 2))

In [ ]:
def plot_2_genes_spatial_prep(adata, gene1, gene2, detection_threshold = 0.5, use_raw = True):
    pairs_d = {(True, True): "Both expressed",
               (True, False): f"{gene1} not {gene2}",
               (False, True): f"{gene2} not {gene1}",
               (False, False): "Neither expressed"}
    if use_raw:
        adata.obs[f"{gene1}_{gene2}"] = [pairs_d[x] for x in zip(adata.raw.to_adata().to_df()[gene1] > detection_threshold, adata.raw.to_adata().to_df()[gene2] > detection_threshold)]
    else:
        adata.obs[f"{gene1}_{gene2}"] = [pairs_d[x] for x in zip(adata.to_df()[gene1] > detection_threshold, adata.to_df()[gene2] > detection_threshold)]
    print(f"{gene1}_{gene2}" + " added to .obs")
    return adata

In [ ]:
def plot_2_gene_programs_spatial_prep(adata, gene_dict1 , gene_dict2, detection_threshold = 0.5, use_raw = True):
    pairs_d = {(True, True): "Both expressed",
               (True, False): f"{list(gene_dict1.keys())[0]} not {list(gene_dict2.keys())[0]}",
               (False, True): f"{list(gene_dict2.keys())[0]} not {list(gene_dict1.keys())[0]}",
               (False, False): "Neither expressed"}
    if use_raw:
        adata.obs[f"{list(gene_dict1.keys())[0]}_{list(gene_dict2.keys())[0]}"] = [pairs_d[x] for x in zip(adata.raw.to_adata().to_df()[list(gene_dict1.values())[0]].sum(axis = 1) > detection_threshold, adata.raw.to_adata().to_df()[list(gene_dict2.values())[0]].sum(axis = 1) > detection_threshold)]
    else:
        adata.obs[f"{list(gene_dict1.keys())[0]}_{list(gene_dict2.keys())[0]}"] = [pairs_d[x] for x in zip(adata.to_df()[list(gene_dict1.values())[0]].sum(axis = 1) > detection_threshold, adata.to_df()[list(gene_dict2.values())[0]].sum(axis = 1) > detection_threshold)]
    print(f"{gene1}_{gene2}" + " added to .obs")
    return adata

In [ ]:
granulosa_temp = plot_2_genes_spatial_prep(granulosa_temp, gene1="Kctd14", gene2="Inhbb", detection_threshold = 0.1, use_raw=False)

# granulosa_mural = plot_2_gene_programs_spatial_prep(granulosa_temp,
#                                                     gene_dict1={"Preantral": ["Kctd14", "Slc18a2"]},
#                                                     gene_dict2={"Antral": ["Inhbb", "Inhba"]}, detection_threshold=2.0)

In [ ]:
sc.tl.rank_genes_groups(granulosa_temp, groupby="Kctd14_Inhbb", key_added="Kctd14_Inhbb_markers", method="wilcoxon")

In [ ]:
result = granulosa_temp.uns['Kctd14_Inhbb_markers']
groups = result['names'].dtype.names
dge_results = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names', "logfoldchanges", "pvals", "pvals_adj"]})

In [ ]:
dge_results.head(20)

In [ ]:
sc.tl.dendrogram(granulosa_temp, groupby="Kctd14_Inhbb")
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=6, facecolor='white', frameon=False, figsize=(1.0, 1.0), vector_friendly = False, transparent=True, format="pdf")
sc.pl.rank_genes_groups_matrixplot(granulosa_temp, key="Kctd14_Inhbb_markers", n_genes=10, cmap='RdBu_r', values_to_plot="logfoldchanges", min_logfoldchange=1.5, vmin = -3, vmax = 3, save = "Kctd_inhbb_subtype_markers", figsize = (6,0.5))

In [ ]:
double_positive_markers = sc.get.rank_genes_groups_df(granulosa_temp, key='Kctd14_Inhbb_markers', group = "Both expressed")
double_positive_markers = double_positive_markers[double_positive_markers["pvals"] < 0.05]
double_positive_markers = double_positive_markers[double_positive_markers["logfoldchanges"] > 1.5]
# double_positive_markers = double_positive_markers.sort_values(by="logfoldchanges", ascending=False)
double_positive_markers.head(30)

In [ ]:
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=8, facecolor='white', frameon=False, figsize=(2.0,2.0), vector_friendly = True, transparent=True, format="pdf")
sc.pl.spatial(granulosa_temp, color=["Ank1", "Aldh3b1", "Irx3", "Cacna1a"], spot_size=30, ncols=2, wspace=0.1, cmap="Blues")

In [ ]:
freq_table = granulosa_temp.obs.groupby(['FollicleNumber', "Follicle_type", "phase"]).size().reset_index(name='count')
freq_table['total_count'] = freq_table.groupby(['FollicleNumber', "Follicle_type"])['count'].transform('sum')
freq_table['proportion'] = freq_table['count'] / freq_table['total_count']
# freq_table["Follicle_type"] = freq_table["Follicle_type"].cat.remove_categories(["Lytic"])
# freq_table["Follicle_type"] = freq_table["Follicle_type"].cat.reorder_categories(["Preantral", "Antral", "Atretic", "Transitioning"])
freq_table = freq_table[freq_table["count"] > 0]
freq_table.head(20)

In [ ]:
sc.settings.set_figure_params(dpi=80, fontsize=6, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
sns.set_style("ticks")
fig, ax = plt.subplots(figsize = (1.8,1.5))
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0, size = 2)
sns.barplot(freq_table, y = "Follicle_type", x = "proportion", dodge=True, hue="phase", palette="Set2", errwidth=1.5, ax=ax)
plt.legend(ncol=1, prop={'size': 6}, handletextpad=0.5, markerscale = 0.6, title = "Cell cycle",
           labelspacing = 0.0, frameon = False,  title_fontsize = 6, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, loc = 1, bbox_to_anchor = (1.0, 1.0))
plt.rcParams['legend.title_fontsize'] = 9
ax.set_xlim(0,1.0)
ax.set_ylabel("Follicle type", labelpad = -2)
ax.set_xlabel("Cell proportions", labelpad = 1.0)
plt.tight_layout()
plt.savefig("./figures/cellcycle_immature.pdf")

In [ ]:
freq_table = granulosa_temp.obs.groupby(['FollicleNumber', "Follicle_type", "Kctd14_Inhbb"]).size().reset_index(name='count')
freq_table['total_count'] = freq_table.groupby(['FollicleNumber', "Follicle_type"])['count'].transform('sum')
freq_table['proportion'] = freq_table['count'] / freq_table['total_count']
# freq_table["Follicle_type"] = freq_table["Follicle_type"].cat.remove_categories(["Lytic"])
# freq_table["Follicle_type"] = freq_table["Follicle_type"].cat.reorder_categories(["Preantral", "Antral", "Atretic", "Transitioning"])
freq_table = freq_table[freq_table["count"] > 0]
freq_table.head(20)

In [ ]:
sc.settings.set_figure_params(dpi=80, fontsize=6, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
sns.set_style("ticks")
fig, ax = plt.subplots(figsize = (1.5,1.5))
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0, size = 2)
sns.barplot(freq_table, y = "Follicle_type", x = "proportion", dodge=True, hue="Kctd14_Inhbb", palette="tab10", ax=ax)
plt.legend(ncol=1, prop={'size': 6}, handletextpad=0.5, markerscale = 0.6, title = "Kctd & Inhbb expression",
           labelspacing = 0.0, frameon = False,  title_fontsize = 7, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, loc = 6, bbox_to_anchor = (1.0, 0.5))
plt.rcParams['legend.title_fontsize'] = 9
ax.set_xlim(0,1.0)
ax.set_ylabel("Follicle type", labelpad = -2)
ax.set_xlabel("Cell proportions", labelpad = 0.5)
ax.get_legend().remove()
plt.tight_layout()
plt.savefig("./figures/kctd14_inhbb_immature.pdf")

In [ ]:
freq_table = granulosa_temp.obs.groupby(['FollicleNumber', "Kctd14_Inhbb", "phase"]).size().reset_index(name='count')
freq_table['total_count'] = freq_table.groupby(['FollicleNumber', "Kctd14_Inhbb"])['count'].transform('sum')
freq_table['proportion'] = freq_table['count'] / freq_table['total_count']
# freq_table["Follicle_type"] = freq_table["Follicle_type"].cat.remove_categories(["Lytic"])
# freq_table["Follicle_type"] = freq_table["Follicle_type"].cat.reorder_categories(["Preantral", "Antral", "Atretic", "Transitioning"])
freq_table = freq_table[freq_table["count"] > 0]
freq_table.head(20)

In [ ]:
sc.settings.set_figure_params(dpi=80, fontsize=6, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
sns.set_style("ticks")
fig, ax = plt.subplots(figsize = (2.0,1.5))
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0, size = 2)
sns.barplot(freq_table, y = "Kctd14_Inhbb", x = "proportion", dodge=True, hue="phase", palette="Set2", ax=ax)
plt.legend(ncol=1, prop={'size': 6}, handletextpad=0.5, markerscale = 0.6, title = "Cell cycle",
           labelspacing = 0.0, frameon = False,  title_fontsize = 6, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, loc = 1, bbox_to_anchor = (1.0, 1.0))
plt.rcParams['legend.title_fontsize'] = 9
ax.set_xlim(0,1.0)
ax.set_ylabel("Kctd & Inhbb expression", labelpad = -2)
ax.set_xlabel("Cell proportions", labelpad = 0.5)
# ax.get_legend().remove()
plt.tight_layout()
plt.savefig("./figures/kctd14_inhbb_phase_immature.pdf")

In [ ]:
granulosa_temp = plot_2_genes_spatial_prep(granulosa_temp, gene1="Kctd14", gene2="Cfh", detection_threshold = 0.1, use_raw=False)

In [ ]:
granulosa_temp.write_h5ad("./../pyobjs/granulosa_mural_immature.h5ad")

In [ ]:
granulosa_temp.obs

In [ ]:
granulosa_mural = sc.read_h5ad("./../pyobjs/granulosa_mural.h5ad")
granulosa_mural.uns['log1p']["base"] = None
granulosa_mural.shape

In [ ]:
granulosa_temp = granulosa_mural[granulosa_mural.obs["Sample"] == "0hr"]

In [ ]:
sc.tl.rank_genes_groups(granulosa_temp, groupby="granulosa_celltypes", groups=["GC_Mural_Outer-(Antral)", "GC_Mural_Inner-(Antral)"], key_added="AO_vs_AI")

In [ ]:
result = granulosa_temp.uns['AO_vs_AI']
groups = result['names'].dtype.names
dge_results = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names', "logfoldchanges", "pvals_adj"]}).head(10)
print(dge_results.head(20))

In [ ]:
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=20, facecolor='white', frameon=False, figsize=(1.0, 1.0), vector_friendly = False, transparent=True, format="pdf")
sc.pl.rank_genes_groups_matrixplot(granulosa_temp, key="AO_vs_AI", n_genes=30, cmap='magma_r', min_logfoldchange=1.0)

In [ ]:
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=8, facecolor='white', frameon=False, figsize=(2.0,2.0), vector_friendly = True, transparent=True, format="pdf")
sc.pl.spatial(granulosa_temp, color=["Bmp3", "Slc38a5", "Ccnd2"], spot_size=30, ncols=3, wspace=0.1, cmap=sns.blend_palette(["lightgrey", sns.xkcd_rgb["darkblue"]], as_cmap=True), vmax = 'p99.2')

In [ ]:
granulosa_temp = granulosa_mural[granulosa_mural.obs["Sample"].isin(["Immature", "0hr"])]
granulosa_temp = granulosa_temp[granulosa_temp.obs["granulosa_celltypes_reduced"].isin(["GC_Mural-(Antral)"])]
granulosa_temp.shape

In [ ]:
sc.tl.rank_genes_groups(granulosa_temp, groupby="Sample", key_added="SAF_vs_LAF")

In [ ]:
# sc.tl.rank_genes_groups(granulosa_temp, groupby="Sample", key_added="SAF_vs_LAF")
result = granulosa_temp.uns['SAF_vs_LAF']
groups = result['names'].dtype.names
dge_results = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names', "logfoldchanges", "pvals_adj"]}).head(10)
print(dge_results.head(30))

In [ ]:
sc.settings.set_figure_params(dpi=300, dpi_save= 300, fontsize=20, facecolor='white', frameon=False, figsize=(1.0, 1.0), vector_friendly = False, transparent=True, format="pdf")
sc.pl.rank_genes_groups_matrixplot(granulosa_temp, key="SAF_vs_LAF", n_genes=15, cmap='magma_r', min_logfoldchange=2.0)
sc.settings.set_figure_params(dpi=80, dpi_save= 300, fontsize=8, facecolor='white', frameon=False, figsize=(3.0, 3.0))


In [ ]:
granulosa_temp = granulosa_mural[granulosa_mural.obs["Sample"].isin(["0hr", "1hr"])]
granulosa_temp = granulosa_temp[granulosa_temp.obs["Follicle_type"] == "Antral"]
granulosa_temp.shape

In [ ]:
sc.tl.rank_genes_groups(granulosa_temp, groupby="Sample", groups=["1hr"], reference="0hr", key_added="early_hcg_chnages", pts=True, method="wilcoxon")

In [ ]:
dge_results = sc.get.rank_genes_groups_df(granulosa_temp, key="early_hcg_chnages", group = "1hr")
f_tfs = "./../../pyscenic_databases/allTFs_mm.txt"   # mouse
tfs = pd.read_csv(f_tfs, header=None)[0].values
dge_results["TFs"] = np.in1d(dge_results.names.values, tfs)
# tf_names = load_tf_names( f_tfs )
# dge_results.to_csv("./../pyobjs/early_hcg_changes.csv")
dge_results

In [ ]:
dge_results = dge_results[dge_results["pvals_adj"] < 0.01]
dge_results = dge_results.sort_values(by = "logfoldchanges", ascending=False)
dge_results = dge_results[dge_results["logfoldchanges"] > 0.5]
print(dge_results.shape)
dge_results.head(20)

In [ ]:
import gseapy
enr_res = gseapy.enrichr(gene_list=list(dge_results["names"]), 
                     organism='Mouse',
                     gene_sets=['GO_Biological_Process_2021'],
#                      description='pathway', 
                     cutoff = 0.01)

In [ ]:
GO_results = enr_res.results
GO_results["-log p-value"] =  -1 * np.log10(GO_results["Adjusted P-value"]) 
GO_results["Term"] = [x.split(" (")[0] for x in GO_results["Term"]]
GO_results[0:15]


In [ ]:
sc.settings.set_figure_params(dpi_save= 400, fontsize=5, facecolor='white', frameon=True, transparent=True, format="pdf")
fig, ax = plt.subplots(figsize=(1.0,1.2))
plt.rc('font', size=5)
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
ax.set_axisbelow(True)
plt.grid(b=None)
g = sns.barplot(y = "Term", x = "-log p-value", data= GO_results[1:7], ax=ax)
from textwrap import wrap
labels = [x.get_text() for x in ax.get_yticklabels()]
labels = ['\n'.join(wrap(l, 30)) for l in labels]
print(labels)
ax.set_yticklabels(labels, rotation=0, horizontalalignment='right')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, horizontalalignment='right')
ax.set_xlabel("-log p-value", labelpad=0.5)
ax.set_ylabel("Term", labelpad=0.0)
plt.xlim((0,4.5))
# plt.tight_layout()
plt.savefig("./figures/early_hcg_go.pdf")

In [ ]:
dge_results = sc.get.rank_genes_groups_df(granulosa_temp, key="early_hcg_chnages", group = "1hr")
f_tfs = "./../../pyscenic_databases/allTFs_mm.txt"   # mouse
tfs = pd.read_csv(f_tfs, header=None)[0].values
dge_results["TFs"] = np.in1d(dge_results.names.values, tfs)
# tf_names = load_tf_names( f_tfs )
# dge_results.to_csv("./../pyobjs/early_hcg_changes.csv")
dge_results

In [ ]:
dge_results = dge_results[dge_results["pvals_adj"] < 0.01]
dge_results = dge_results[dge_results["TFs"] == True]
dge_results = dge_results.sort_values(by = "scores", ascending=False)
dge_results.head(10)

In [ ]:
# dge_results.to_csv("./../pyobjs/early_hcg_changes_tfs.csv")

In [ ]:
sc.settings.set_figure_params(dpi=300, dpi_save = 300, fontsize=6, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = True, transparent=True, format="pdf")
sns.set_style("ticks")
fig, axs = plt.subplots(2,3, figsize = (2.1,1.4))
axs = axs.flatten()
gene_list = dge_results.head(6)["names"]
data = granulosa_temp.obs.join(granulosa.to_df()[gene_list])
for i, gene in enumerate(gene_list):
    print(i, gene)
    sns.boxplot(data, y=gene, x="Sample", ax = axs[i], linewidth=1.0, fliersize=1.0)
#     if i == 0:
#         legend = axs[i].get_legend()
#     axs[i].get_legend().remove()
    axs[i].tick_params(axis='both', which='major', pad=0, size = 2)
    axs[i].set_ylabel(gene, labelpad = 0.0)
    axs[i].set_xlabel("", labelpad = 0.0) 
# plt.legend(ncol=1, prop={'size': 6}, handletextpad=0.5, markerscale = 0.6, title = "Kctd & Inhbb expression",
#            labelspacing = 0.0, frameon = False,  title_fontsize = 7, alignment = "left", 
#            borderpad = 0.1, borderaxespad = 0.1, loc = 6, bbox_to_anchor = (1.0, 0.5))     
plt.subplots_adjust(wspace=0, hspace=0.5)
plt.tight_layout()
plt.savefig("./figures/early_hcg_tfs.pdf", dpi = 300)

In [ ]:
granulosa_mural = sc.read_h5ad("./../../pyobjs_github/granulosa_mural.h5ad")
granulosa_mural.uns['log1p']["base"] = None
granulosa_mural_subset = granulosa_mural[~granulosa_mural.obs["Follicle_type"].isin(["Antral"])]
granulosa_mural_subset = granulosa_mural_subset[~granulosa_mural_subset.obs["Sample"].isin(["Immature"])]
granulosa_mural_subset.shape

In [ ]:
granulosa_mural_subset.obs["timepoint"] = granulosa_mural_subset.obs["Sample"]
celltypes = {'0hr': 0, 
            '1hr': 1, 
            '4hr': 4, 
            '6hr': 6, 
            '8hr': 8,
            '11hr': 11,
            '12hr': 12}
granulosa_mural_subset.obs["timepoint"] = granulosa_mural_subset.obs["timepoint"].map(celltypes)
granulosa_mural_subset.obs["timepoint"] = granulosa_mural_subset.obs["timepoint"].astype(int)

In [ ]:
sc.pp.highly_variable_genes(granulosa_mural_subset, min_disp=0.5, min_mean=0.2)
sc.pl.highly_variable_genes(granulosa_mural_subset)
gene_list = list(granulosa_mural_subset.var_names[granulosa_mural_subset.var.highly_variable])
len(gene_list)

In [ ]:
mural_path = ['GC_Mural-(Antral)']
trend_results = scGAM.compute_gene_trends(granulosa_mural_subset, var_key="timepoint", genes = gene_list, n_bins = 0, use_raw = False, lineages_paths={"mural":mural_path}, group_by = "granulosa_celltypes_reduced")

In [ ]:
trends = trend_results['mural']['trend']

In [ ]:
trends.to_csv("./../pyobjs/mural_granulosa_gene_trends_timepoint_discrete.csv")

In [ ]:
clusters = scGAM.Agglomerative_cluster_gene_trends_from_lineage(trends, n_clusters=5, scale=True)
clusters.value_counts()

In [ ]:
# def plot_gene_expression_trend_clusters(trends, cluster_ids, scale = True):
#     if scale: 
#         trends_df = pd.DataFrame(
#             StandardScaler().fit_transform(trends.T).T,
#             index=trends.index,
#             columns=trends.columns,
#         ).T
#     else:
#         trends_df = trends.T
#     trends_df = trends_df.stack().reset_index().rename(columns = {"level_0": "timepoint", "level_1":"Gene", 0: "Expression"})
#     trends_df["Cluster"] = [cluster_ids[x] for x in trends_df["Gene"]]
#     return trends_df

# def plot_gene_expression_trends(trends, genes, scale = True):
#     if scale: 
#         trends_df = pd.DataFrame(
#             StandardScaler().fit_transform(trends.T).T,
#             index=trends.index,
#             columns=trends.columns,
#         ).T[genes]
#     else:
#         trends_df = trends.T[genes]
#     trends_df = trends_df.stack().reset_index().rename(columns = {"level_0": "timepoint", "level_1":"Gene", 0: "Expression"})
#     return trends_df

In [ ]:
trends_df = plot_gene_expression_trend_clusters(trends = trends, cluster_ids=clusters, scale=True)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=200, fontsize=6, facecolor='white', frameon=True, figsize=(1.5, 1.5))
fig, ax = plt.subplots(figsize = (2.3,1.5))
plt.grid(False)
sns.set_style("ticks")
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
g = sns.lineplot(data = trends_df, x = "timepoint", y= "Expression", hue="Cluster", palette="tab10", lw = 1, ax=ax)
g.set_xticks(range(0, 13, 2))
plt.legend(ncol=1, prop={'size': 6}, loc = 2, handletextpad=0.5, markerscale = 0.6, title = "Gene groups", 
           labelspacing = 0.0, frameon = False, title_fontsize = 7, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, bbox_to_anchor = (1.1, 0.75))
ax.get_legend().set_title("Gene groups")
ax.set_xlabel("Time after LH treatment", labelpad=0.5)
ax.set_ylabel("Scaled expression", labelpad=0.0)
plt.tight_layout()

In [ ]:
trends_df = plot_gene_expression_trends(trends = trends, genes=granulosa_mural_subset.var.loc[clusters[clusters == 3].index.values,:].sort_values(by = "dispersions_norm", ascending = False).index.values[0:15], scale=True)

In [ ]:
sc.settings.set_figure_params(dpi=200, fontsize=7, facecolor='white', frameon=True, figsize=(5, 5))
fig, ax = plt.subplots(figsize = (2.5,1.5))
plt.grid(False)
sns.set_style("ticks")
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
g = sns.lineplot(data = trends_df, x = "timepoint", y= "Expression", hue="Gene", palette="Dark2", lw = 1, ax=ax)
g.set_xticks(range(0, 13, 2))
plt.legend(ncol=1, prop={'size': 6}, loc = 6, handletextpad=0.5, markerscale = 0.6, title = "Gene groups", 
           labelspacing = 0.0, frameon = False, title_fontsize = 6, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, bbox_to_anchor = (1.05, 0.50))
ax.get_legend().set_title("Genes")
ax.set_xlabel("Time after LH treatment", labelpad=0.5)
ax.set_ylabel("Scaled expression", labelpad=0.0)
plt.tight_layout()
# fig.savefig("./figures/increasing_trends.pdf")

In [ ]:
# #Available databases : ‘Human’, ‘Mouse’, ‘Yeast’, ‘Fly’, ‘Fish’, ‘Worm’ 
# gene_set_names = gseapy.get_library_name(organism='Mouse')
# print(gene_set_names)

In [ ]:
# # Some databases of interest:
# GO_Biological_Process_2021
# GO_Biological_Process_2017b
# KEGG_2019_Human
# KEGG_2019_Mouse
# WikiPathways_2019_Human
# WikiPathways_2019_Mouse

In [ ]:
# import gseapy
# enr_res = gseapy.enrichr(gene_list=list(clusters[clusters == 3].index), 
#                      organism='Mouse',
#                      gene_sets=['GO_Biological_Process_2021'],
# #                      description='pathway', 
#                      cutoff = 0.5)

In [ ]:
# GO_results = enr_res.results
# GO_results["-log p-value"] =  -1 * np.log10(GO_results["Adjusted P-value"]) 
# GO_results["Term"] = [x.split(" (")[0] for x in GO_results["Term"]]
# GO_results[0:5]

In [ ]:
# sc.settings.set_figure_params(dpi_save= 400, fontsize=5, facecolor='white', frameon=True, transparent=True, format="pdf")
# fig, ax = plt.subplots(figsize=(0.9,1.5))
# plt.rc('font', size=5)
# plt.margins(x=0, y=0)
# ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
# ax.set_axisbelow(True)
# plt.grid(b=None)
# g = sns.barplot(y = "Term", x = "-log p-value", data= GO_results[0:10], ax=ax)
# # from textwrap import wrap
# # labels = [x.get_text() for x in ax.get_yticklabels()]
# # labels = ['\n'.join(wrap(l, 40)) for l in labels]
# # # print(labels)
# # ax.set_yticklabels(labels, rotation=0, horizontalalignment='right')
# ax.set_xticklabels(ax.get_xticklabels(), rotation=0, horizontalalignment='right')
# ax.set_xlabel("-log p-value", labelpad=0.5)
# ax.set_ylabel("Term", labelpad=0.0)
# plt.tight_layout()

In [ ]:
granulosa_mural_bulk = sc.read_h5ad("./../../pyobjs_github/granulosa_mural_bulk_raw_counts.h5ad")
# granulosa_mural_bulk.uns['log1p']["base"] = None
granulosa_mural_bulk

In [ ]:
granulosa_mural_bulk[granulosa_mural_bulk.obs["Sample"] == "12hr"].obs["Follicle_type"].value_counts()

In [ ]:
sc.pp.normalize_total(granulosa_mural_bulk, target_sum=1e4)
sc.pp.log1p(granulosa_mural_bulk)

In [ ]:
granulosa_mural_bulk_subset = granulosa_mural_bulk[~granulosa_mural_bulk.obs["Sample"].isin(["Immature"])]
granulosa_mural_bulk_subset.shape

In [ ]:
granulosa_mural_bulk_subset.obs["Follicle_type"].value_counts()

In [ ]:
granulosa_mural_bulk_subset = granulosa_mural_bulk_subset[granulosa_mural_bulk_subset.obs["Follicle_type"].isin(["Antral"])]
granulosa_mural_bulk_subset.shape

In [ ]:
sc.pp.highly_variable_genes(granulosa_mural_bulk_subset, min_disp=0.5, min_mean=0.5)
sc.pl.highly_variable_genes(granulosa_mural_bulk_subset)
gene_list = list(granulosa_mural_bulk_subset.var_names[granulosa_mural_bulk_subset.var.highly_variable])
len(gene_list)

In [ ]:
"Edn2" in gene_list

In [ ]:
granulosa_mural_bulk_subset.obs["timepoint"] = granulosa_mural_bulk_subset.obs["Sample"]
celltypes = {'Immature': -1, 
             '0hr': 0, 
            '1hr': 1, 
            '4hr': 4, 
            '6hr': 6, 
            '8hr': 8,
            '11hr': 11,
            '12hr': 12}
granulosa_mural_bulk_subset.obs["timepoint"] = granulosa_mural_bulk_subset.obs["timepoint"].map(celltypes)
granulosa_mural_bulk_subset.obs["timepoint"] = granulosa_mural_bulk_subset.obs["timepoint"].astype(np.int_)

In [ ]:
antral_mural_path = ['Antral']
trend_results = scGAM.compute_gene_trends(granulosa_mural_bulk_subset, var_key="timepoint", genes = gene_list, n_bins = 0, use_raw = False, lineages_paths={"Antral": antral_mural_path}, group_by = "Follicle_type")

In [ ]:
trends = trend_results['Antral']['trend']
trends.shape

In [ ]:
trends.to_csv("./../pyobjs/follicle_muralgranulosa_gene_trends_timepoint_discrete.csv")

In [ ]:
trends= pd.read_csv("./../pyobjs/follicle_muralgranulosa_gene_trends_timepoint_discrete.csv", index_col=0)
trends

In [ ]:
clusters = scGAM.Agglomerative_cluster_gene_trends_from_lineage(trends, n_clusters=6, scale=True)
clusters.value_counts()

In [ ]:
trends_df = plot_gene_expression_trend_clusters(trends = trends, cluster_ids=clusters, scale=True)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=200, fontsize=6, facecolor='white', frameon=True, figsize=(1.5, 1.5))
fig, ax = plt.subplots(figsize = (2.3,1.5))
plt.grid(False)
sns.set_style("ticks")
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
g = sns.lineplot(data = trends_df, x = "timepoint", y= "Expression", hue="Cluster", palette="tab10", lw = 1, ax=ax)
g.set_xticks([0,2,4,6])
plt.legend(ncol=1, prop={'size': 6}, loc = 2, handletextpad=0.5, markerscale = 0.6, title = "Gene groups", 
           labelspacing = 0.0, frameon = False, title_fontsize = 6, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, bbox_to_anchor = (1.05, 0.75))
ax.get_legend().set_title("Gene groups")
ax.set_xlabel("Time after hCG treatment", labelpad=0.5)
ax.set_ylabel("Scaled expression", labelpad=0.0)
plt.tight_layout()
fig.savefig("./figures/mural _granulosa_gene_group_trends.pdf")
# plt.show()

In [ ]:
trends_df = plot_gene_expression_trends(trends = trends, genes=granulosa_mural_bulk_subset.var.loc[clusters[clusters == 0].index.values,:].sort_values(by = "dispersions_norm", ascending = False).index.values[0:10], scale=True)

In [ ]:
sc.settings.set_figure_params(dpi=200, fontsize=6, facecolor='white', frameon=True, figsize=(5, 5))
fig, ax = plt.subplots(figsize = (2.3,1.5))
plt.grid(False)
sns.set_style("ticks")
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
g = sns.lineplot(data = trends_df, x = "timepoint", y= "Expression", hue="Gene", palette="Dark2", lw = 1, ax=ax)
g.set_xticks([0,2,4,6])
plt.legend(ncol=1, prop={'size': 6}, loc = 6, handletextpad=0.5, markerscale = 0.6, title = "Gene groups", 
           labelspacing = 0.0, frameon = False, title_fontsize = 6, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, bbox_to_anchor = (1.05, 0.50))
ax.get_legend().set_title("Genes")
ax.set_xlabel("Time after hCG treatment", labelpad=0.5)
ax.set_ylabel("Scaled expression", labelpad=0.0)
plt.tight_layout()
fig.savefig("./figures/decreasing_trends.pdf")

In [ ]:
import gseapy
enr_res = gseapy.enrichr(gene_list=list(clusters[clusters == 0].index), 
                     organism='Mouse',
                     gene_sets=['GO_Biological_Process_2021'],
#                      description='pathway', 
                     cutoff = 0.01)

In [ ]:
GO_results = enr_res.results
GO_results["-log p-value"] =  -1 * np.log10(GO_results["Adjusted P-value"]) 
GO_results["Term"] = [x.split(" (")[0] for x in GO_results["Term"]]
GO_results[0:10]

In [ ]:
GO_results[GO_results["Term"] == "negative regulation of apoptotic process"]

In [ ]:
apoptosis_genes = GO_results[GO_results["Term"] == "negative regulation of apoptotic process"].Genes.values[0].split(";")
from gseapy import Biomart
bm = Biomart()
h2m = bm.query(dataset='hsapiens_gene_ensembl',
               attributes=['external_gene_name',
                           'mmusculus_homolog_associated_gene_name'])
h2m.index = h2m["external_gene_name"]
apoptosis_genes = h2m.loc[apoptosis_genes]["mmusculus_homolog_associated_gene_name"]
apoptosis_genes = apoptosis_genes[~pd.isna(apoptosis_genes)]

In [ ]:
apoptosis_genes

In [ ]:
apoptosis_genes.to_csv("./../pyobjs/apoptosis_genes.csv")

In [ ]:
sc.settings.set_figure_params(dpi_save= 400, fontsize=5, facecolor='white', frameon=True, transparent=True, format="pdf")
fig, ax = plt.subplots(figsize=(1.2,1.5))
plt.rc('font', size=6)
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
ax.set_axisbelow(True)
plt.grid(b=None)
g = sns.barplot(y = "Term", x = "-log p-value", data= GO_results[0:15], ax=ax)
# from textwrap import wrap
# labels = [x.get_text() for x in ax.get_yticklabels()]
# labels = ['\n'.join(wrap(l, 40)) for l in labels]
# print(labels)
# ax.set_yticklabels(labels, rotation=0, horizontalalignment='right')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, horizontalalignment='right')
ax.set_xlabel("-log p-value", labelpad=0.5)
ax.set_ylabel("Term", labelpad=0.0)
# plt.tight_layout()
plt.savefig("./figures/decreasing_trends_go.pdf")

In [ ]:
trends_df = plot_gene_expression_trends(trends = trends, genes=granulosa_mural_bulk_subset.var.loc[clusters[clusters == 2].index.values,:].sort_values(by = "dispersions_norm", ascending = False).index.values[0:10], scale=True)

In [ ]:
sc.settings.set_figure_params(dpi=200, fontsize=6, facecolor='white', frameon=True, figsize=(5, 5))
fig, ax = plt.subplots(figsize = (2.3,1.5))
plt.grid(False)
sns.set_style("ticks")
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
g = sns.lineplot(data = trends_df, x = "timepoint", y= "Expression", hue="Gene", palette="Dark2", lw = 1, ax=ax)
g.set_xticks([0,2,4,6])
plt.legend(ncol=1, prop={'size': 6}, loc = 6, handletextpad=0.5, markerscale = 0.6, title = "Gene groups", 
           labelspacing = 0.0, frameon = False, title_fontsize = 6, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, bbox_to_anchor = (1.05, 0.50))
ax.get_legend().set_title("Genes")
ax.set_xlabel("Time after hCG treatment", labelpad=0.5)
ax.set_ylabel("Scaled expression", labelpad=0.0)
plt.tight_layout()
fig.savefig("./figures/increasing_trends.pdf")

In [ ]:
import gseapy
enr_res = gseapy.enrichr(gene_list=list(granulosa_mural_bulk_subset.var.loc[clusters[clusters == 2].index.values,:].sort_values(by = "dispersions_norm", ascending = False).index.values), 
                     organism='Mouse',
                     gene_sets=['GO_Biological_Process_2021'],
#                      description='pathway', 
                     cutoff = 0.01)

In [ ]:
GO_results = enr_res.results
GO_results["-log p-value"] =  -1 * np.log10(GO_results["Adjusted P-value"]) 
GO_results["Term"] = [x.split(" (")[0] for x in GO_results["Term"]]
GO_results[0:15]

In [ ]:
sc.settings.set_figure_params(dpi_save= 400, fontsize=5, facecolor='white', frameon=True, transparent=True, format="pdf")
fig, ax = plt.subplots(figsize=(1.2,1.5))
plt.rc('font', size=6)
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
ax.set_axisbelow(True)
plt.grid(b=None)
g = sns.barplot(y = "Term", x = "-log p-value", data= GO_results[0:15], ax=ax)
# from textwrap import wrap
# labels = [x.get_text() for x in ax.get_yticklabels()]
# labels = ['\n'.join(wrap(l, 40)) for l in labels]
# print(labels)
# ax.set_yticklabels(labels, rotation=0, horizontalalignment='right')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, horizontalalignment='right')
ax.set_xlabel("-log p-value", labelpad=0.5)
ax.set_ylabel("Term", labelpad=0.0)
# plt.tight_layout()
plt.savefig("./figures/increasing_trends_go.pdf")

In [ ]:
trends_df = plot_gene_expression_trends(trends = trends, genes=granulosa_mural_bulk_subset.var.loc[clusters[clusters == 3].index.values,:].sort_values(by = "dispersions_norm", ascending = False).index.values[0:10], scale=True)

In [ ]:
sc.settings.set_figure_params(dpi=200, fontsize=7, facecolor='white', frameon=True, figsize=(5, 5))
fig, ax = plt.subplots(figsize = (2.3,1.5))
plt.grid(False)
sns.set_style("ticks")
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0.5, size = 2)
g = sns.lineplot(data = trends_df, x = "timepoint", y= "Expression", hue="Gene", palette="Dark2", lw = 1, ax=ax)
g.set_xticks([0,2,4,6])
plt.legend(ncol=1, prop={'size': 6}, loc = 6, handletextpad=0.5, markerscale = 0.6, title = "Gene groups", 
           labelspacing = 0.0, frameon = False, title_fontsize = 7, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, bbox_to_anchor = (1.05, 0.50))
ax.get_legend().set_title("Genes")
ax.set_xlabel("Time after LH treatment", labelpad=0.5)
ax.set_ylabel("Scaled expression", labelpad=0.0)
plt.tight_layout()
# fig.savefig("./figures/granulosa_late_increasing_trends.pdf")

In [ ]:
import gseapy
enr_res = gseapy.enrichr(gene_list=list(granulosa_mural_bulk_subset.var.loc[clusters[(clusters == 3) | (clusters == 3)].index.values,:].sort_values(by = "dispersions_norm", ascending = False).index.values), 
                     organism='Mouse',
                     gene_sets=['GO_Biological_Process_2021'],
#                      description='pathway', 
                     cutoff = 0.01)

In [ ]:
GO_results = enr_res.results
GO_results["-log p-value"] =  -1 * np.log10(GO_results["Adjusted P-value"]) 
GO_results["Term"] = [x.split(" (")[0] for x in GO_results["Term"]]
GO_results[0:15]

In [ ]:
freq_table = granulosa_mural_subset.obs.groupby(['FollicleNumber', "Sample", "phase"]).size().reset_index(name='count')
freq_table['total_count'] = freq_table.groupby(['FollicleNumber', "Sample"])['count'].transform('sum')
freq_table['proportion'] = freq_table['count'] / freq_table['total_count']
freq_table = freq_table[freq_table["count"] > 0]
freq_table.head(20)

In [ ]:
sc.settings.set_figure_params(dpi=80, fontsize=6, facecolor='white', frameon=False, figsize=(2.0, 2.0), vector_friendly = False, transparent=True, format="pdf")
sns.set_style("ticks")
fig, ax = plt.subplots(figsize = (2.8,1.5))
plt.margins(x=0, y=0)
ax.tick_params(axis='both', which='major', pad=0, size = 2)
sns.barplot(freq_table, x = "Sample", y = "proportion", dodge=True, hue="phase", palette="Set2", ax=ax)
plt.legend(ncol=1, prop={'size': 6}, handletextpad=0.5, markerscale = 0.6, title = "Cell cycle",
           labelspacing = 0.0, frameon = False,  title_fontsize = 7, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, loc = 1, bbox_to_anchor = (1.0, 1.0))
plt.rcParams['legend.title_fontsize'] = 9
ax.set_ylim(0,1.0)
ax.set_ylabel("Follicle type", labelpad = 0)
ax.set_xlabel("Cell proportions", labelpad = 1.0)
plt.tight_layout()
plt.savefig("./figures/cellcycle_all_granulosa_mural.pdf")

In [ ]:
adata_magic = pd.read_csv("./../../../pyobjs/adata_ovary_combined_magic_edn2_subset.csv", index_col=0)
adata_magic.index = [x.replace("4hr_replicate", "4hr") for x in adata_magic.index]
dremi_df = pd.read_csv("./../../../pyobjs/dremi_magic_Edn2.csv", index_col=0)

In [ ]:
print(dremi_df.loc["Edn2",])
dremi_df = dremi_df[dremi_df["means"] > 0.1]
dremi_df = dremi_df[dremi_df["dispersions"] > 0.1]

In [ ]:
adata_magic.shape

In [ ]:
adata_magic["Sample"] = adata_ovary_combined.obs["Sample"]

In [ ]:
sc.settings.set_figure_params(dpi=80, dpi_save = 300, fontsize=6, facecolor='white', frameon=False, figsize=(1.0, 1.0), vector_friendly = True, transparent=True, format="pdf")
sns.set_style("ticks")
fig, axs = plt.subplots(2,2, figsize = (1.8,1.5))
axs = axs.flatten()
x_gene = "Edn2"
y_gene_list = dremi_df["gene"][1:5]
for i, y_gene in enumerate(y_gene_list):
    print(i, y_gene)
    sns.scatterplot(adata_magic.loc[adata_ovary_combined.obs_names], x="Edn2", y = y_gene, hue = "Sample", palette="Set2", ax=axs[i], 
                    s = 0.5, alpha = 0.5, linewidths = 0.0)
    if i == 0:
        handles, labels = axs[i].get_legend_handles_labels()
        legend = axs[i].get_legend()
    axs[i].get_legend().remove()
    axs[i].tick_params(axis='both', which='major', pad=0, size = 2)
    axs[i].set_ylabel(y_gene, labelpad = 0.0, fontstyle = "italic")
    axs[i].set_xlabel(x_gene, labelpad = 0.0, fontstyle = "italic")
    
fig.legend(handles, labels, ncol=1, prop={'size': 6}, handletextpad=0.5, markerscale = 0.6, title = "Sample",
           labelspacing = 0.0, frameon = False,  title_fontsize = 7, alignment = "left", 
           borderpad = 0.1, borderaxespad = 0.1, loc = 6, bbox_to_anchor = (1.0, 0.5))     
# fig.delaxes(axs[0])
plt.margins(x=0, y=0)
plt.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout()
plt.savefig("./figures/edn2_dremi_coor.png", dpi = 300)

In [ ]:
ovary_11hr = sc.read_h5ad("./../pyobjs/11hr_ovary.h5ad")
ovary_12hr = sc.read_h5ad("./../pyobjs/12hr_ovary.h5ad")

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save= 300, fontsize=8, facecolor='white', frameon=False, figsize=(2.0,2.0), vector_friendly = True, transparent=True, format="pdf")
sc.pl.spatial(ovary_12hr, color=["granulosa_clusters"], spot_size=30, ncols=3, wspace=0.1)

In [ ]:
sample = "12hr"
celltypes = {'0': "GC_Mural-(Antral)", 
             '1': "GC_Lytic-1", 
            '2': "GC_Lytic-2", 
            '3': "GC_Mural-(Antral)", 
            '4': "GC_Mural-(Antral)", 
            '5': "GC_Mural-(Atretic)", 
            '6': pd.NA,
            '7': "GC_Mural-(Preantral)", 
            '8': "GC_Cumulus"}
ovary_12hr.obs["granulosa_celltypes"] = ovary_12hr.obs["granulosa_clusters"].map(celltypes).astype("category")

In [ ]:
from pylab import *
cmap = cm.get_cmap('tab20')
palette_custom = [matplotlib.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]
cmap
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=True, figsize=(3, 3), format="pdf")
sc.pl.spatial(ovary_12hr, color=["granulosa_celltypes"], spot_size=30, frameon=False, wspace=0.3, na_in_legend=False, title="Granulosa celltypes", save="granulosa_celltypes_12hr")

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=True, figsize=(0.8,0.8), format="pdf")
fig, axs = plt.subplots(2,2, figsize = (2.3,1.8))
axs = axs.flatten()
for i, gene in enumerate(["Edn2", "Ptgs1", "Junb", "Adamts1"]):
    sc.pl.spatial(adata_ovary_combined[adata_ovary_combined.obs["Sample"] == "12hr"], color=gene, spot_size=30, frameon=False, wspace=0.1, ncols=5, cmap=sns.blend_palette(["lightgrey", sns.xkcd_rgb["black"]], as_cmap=True), ax=axs[i], vmax = "p99.2", show=False)
    axs[i].set_title(axs[i].get_title(), pad=0)
for i in range(1,5):
    mappable = fig.axes[-i].collections[1]
    colorBar = fig.axes[-i]
    colorBar.remove()
    plt.colorbar(mappable=mappable, ax=axs[-i],
                 fraction=0.25, aspect=7, location = "right",
                 pad=0 ,shrink = 0.3)
for i in range(1,5):
    fig.axes[-i].tick_params(length = 2, pad = 0.0)
    
plt.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout()
# plt.show()
plt.savefig("./figures/rupture_asscoiated.pdf")

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=True, figsize=(0.8,0.8), format="pdf")
fig, axs = plt.subplots(1, 3, figsize = (3.0,1.0))
axs = axs.flatten()
for i, gene in enumerate(["Sfrp4", "Gm2a", "Ltbp1"]):
    sc.pl.spatial(adata_ovary_combined[adata_ovary_combined.obs["Sample"] == "12hr"], color=gene, spot_size=30, frameon=False, wspace=0.1, ncols=5, cmap=sns.blend_palette(["lightgrey", sns.xkcd_rgb["black"]], as_cmap=True), ax=axs[i], vmin = "p1.0", vmax = "p99.2", show=False)
    axs[i].set_title(axs[i].get_title(), pad=0)
for i in range(1,4):
    mappable = fig.axes[-i].collections[1]
    colorBar = fig.axes[-i]
    colorBar.remove()
    plt.colorbar(mappable=mappable, ax=axs[-i],
                 fraction=0.25, aspect=7, location = "right",
                 pad=0 ,shrink = 0.3)
for i in range(1,4):
    fig.axes[-i].tick_params(length = 2, pad = 0.0)
    
plt.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout()
# plt.show()
plt.savefig("./figures/12hr_lytic_asscoiated.pdf")

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=True, figsize=(0.8,0.8), format="pdf")
fig, axs = plt.subplots(1, 3, figsize = (3.0,1.0))
axs = axs.flatten()
for i, gene in enumerate(["Rhox8", "Star", "Ctsl"]):
    sc.pl.spatial(adata_ovary_combined[adata_ovary_combined.obs["Sample"] == "12hr"], color=gene, spot_size=30, frameon=False, wspace=0.1, ncols=5, cmap=sns.blend_palette(["lightgrey", sns.xkcd_rgb["black"]], as_cmap=True), ax=axs[i], vmin = "p1.0", vmax = "p99.2", show=False)
    axs[i].set_title(axs[i].get_title(), pad=0)
for i in range(1,4):
    mappable = fig.axes[-i].collections[1]
    colorBar = fig.axes[-i]
    colorBar.remove()
    plt.colorbar(mappable=mappable, ax=axs[-i],
                 fraction=0.25, aspect=7, location = "right",
                 pad=0 ,shrink = 0.3)
for i in range(1,4):
    fig.axes[-i].tick_params(length = 2, pad = 0.0)
    
plt.subplots_adjust(wspace=0, hspace=0)
plt.tight_layout()
# plt.show()
plt.savefig("./figures/12hr_antral_asscoiated.pdf")

In [ ]:
sc.tl.rank_genes_groups(ovary_12hr, groupby="granulosa_celltypes", groups=["GC_Lytic-1"], reference = "GC_Lytic-2", method="wilcoxon", key_added="between_lytic_diff")
sc.tl.rank_genes_groups(ovary_12hr, groupby="granulosa_celltypes", groups=["GC_Lytic-1"], reference = "GC_Mural-(Antral)", method="wilcoxon", key_added="between_lytic_and_mural_diff")

In [ ]:
result = ovary_12hr.uns['between_lytic_diff']
groups = result['names'].dtype.names
dge_results = pd.DataFrame(
    {key: result[key][group]
    for group in groups for key in ['names', "logfoldchanges", "pvals_adj"]})
print(dge_results)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=False, figsize=(2, 2), format="pdf")
sc.pl.spatial(ovary_12hr, color=dge_results.head(10)["names"], spot_size=30, cmap="viridis", ncols=5, wspace = 0.0)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=False, figsize=(2, 2), format="pdf")
sc.pl.spatial(ovary_12hr, color=dge_results.tail(10)["names"], spot_size=30, cmap="viridis", ncols=5, wspace = 0.0)

In [ ]:
result = ovary_12hr.uns['between_lytic_and_mural_diff']
groups = result['names'].dtype.names
dge_results = pd.DataFrame(
    {key: result[key][group]
    for group in groups for key in ['names', "logfoldchanges", "pvals_adj"]})
print(dge_results)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=False, figsize=(2, 2), format="pdf")
sc.pl.spatial(ovary_12hr, color=dge_results.head(10)["names"], spot_size=30, cmap="viridis", ncols=5, wspace = 0.0)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=False, figsize=(2, 2), format="pdf")
sc.pl.spatial(ovary_12hr, color=dge_results.tail(10)["names"], spot_size=30, cmap="viridis", ncols=5, wspace = 0.0)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=False, figsize=(2, 2), format="pdf")
sc.pl.spatial(ovary_12hr, color=["Edn2", "Adamts1", "Ldhd", "Apln", "Akr1c18"], spot_size=30, cmap="viridis", ncols=5, wspace = 0.0)

In [ ]:
sc.settings.set_figure_params(dpi=200, dpi_save=300, facecolor='white', fontsize=6, frameon=False, figsize=(2, 2), format="pdf")
sc.pl.spatial(ovary_11hr, color=["Edn2", "Adamts1", "Ldhd", "Akr1c18"], spot_size=30, cmap="viridis", ncols=5, wspace = 0.0)